In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
train=pd.read_csv("../input/g-research-crypto-forecasting/train.csv")
suptrain=pd.read_csv("../input/g-research-crypto-forecasting/supplemental_train.csv")
extest=pd.read_csv("../input/g-research-crypto-forecasting/example_test.csv")
exsub=pd.read_csv("../input/g-research-crypto-forecasting/example_sample_submission.csv")
assdet=pd.read_csv("../input/g-research-crypto-forecasting/asset_details.csv")
test=pd.read_csv("../input/g-research-crypto-forecasting/example_test.csv")

In [ ]:
train.head()

In [ ]:
suptrain.head()

In [ ]:
assdet.head()

In [ ]:
assdet["weight-Percent"] = (assdet["Weight"] / assdet["Weight"].sum()) * 100
assdet.sort_values("Weight", ascending=False)
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
cmap = sns.color_palette()
fig = px.bar(assdet, y='weight-Percent', x='Asset_Name', title='Weight_Percent/Cryptography')
fig.show()

In [ ]:
asset_ID=1
tgt_df = train[train["Asset_ID"] == asset_ID]
tgt_df = tgt_df[['Open', 'High', 'Low', 'Close', 'Volume','timestamp']]  ## ohlcvと時間の情報のみ必要。
tgt_df["timestamp"] = pd.to_datetime(tgt_df['timestamp'].astype(int), unit='s')
tgt_df.set_index("timestamp",inplace=True)

In [ ]:
tgt_df.head()

In [ ]:
train.isnull().sum()

In [ ]:
train=train.dropna()

In [ ]:
train.describe()

In [ ]:
train.corr()

In [ ]:
import numpy as np
from lightgbm import LGBMRegressor
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']
def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['Upper_Shadow'] = upper_shadow(df_feat)
    df_feat['Lower_Shadow'] = lower_shadow(df_feat)
    return df_feat

def get_Xy_and_model_for_asset(train, asset_id):
    df = train[train["Asset_ID"] == asset_id]
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]

    model = LGBMRegressor(n_estimators=1000)
    model.fit(X, y)
    return X, y, model

In [ ]:
Xs = {}
ys = {}
models = {}

for asset_id, asset_name in zip(assdet['Asset_ID'], assdet['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    X, y, model = get_Xy_and_model_for_asset(train, asset_id)    
    Xs[asset_id], ys[asset_id], models[asset_id] = X, y, model

In [ ]:
x = get_features(train.iloc[1])
y_pred = models[0].predict([x])
y_pred[0]

In [ ]:
import gresearch_crypto
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (test, df_pred) in enumerate(iter_test):
    for j , row in test.iterrows():
        
        model = models[row['Asset_ID']]
        x_test = get_features(row)
        y_pred = model.predict([x_test])[0]
        
        df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
        
        
        # Print just one sample row to get a feeling of what it looks like
        if i == 0 and j == 0:
            display(x_test)

    # Display the first prediction dataframe
    if i == 0:
        display(df_pred)

    # Send submissions
    env.predict(df_pred)

# Work in progress, please upvote